In [1]:
# Import of libraries

import nibabel as nib
import numpy as np
import os
import glob

from nilearn.image import resample_img
from skimage.measure import label
from scipy.ndimage.morphology import binary_dilation

In [2]:
# *Exlucsively for explanation*
import matplotlib.pyplot as plt

In [3]:
# Function selecting a bounding box (with a margin of choice) around optic chiasm mask created by FreeSurfer

def bounding_box(img, x_offset, y_offset, z_offset):

    a = np.any(img, axis=(1, 2))
    b = np.any(img, axis=(0, 2))
    c = np.any(img, axis=(0, 1))

    amin, amax = np.where(a)[0][[0, -1]]
    bmin, bmax = np.where(b)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]

    return amin-x_offset, amax+x_offset, bmin-y_offset, bmax+y_offset, cmin-z_offset, cmax+z_offset

In [4]:
# Function visualization the process by showing the mask overlaid on T1w images axial slices
def view_compare(img1, img2, start_slice, number_slices, x_s, x_e, y_s, y_e, alone=False, overlaid=False):

    fig = plt.figure(figsize=(50,50))
    
    if alone:
            for i in range(number_slices):
                fig.add_subplot(number_slices/3,3,i+1)
                plt.imshow(img1[x_s:x_e,y_s:y_e,start_slice+i], cmap='gray')
                plt.title(str(i), fontsize=40)
    
    elif overlaid:
            for i in range(number_slices):
                fig.add_subplot(number_slices/3,3,i+1)
                plt.imshow(img1[x_s:x_e,y_s:y_e,start_slice+i], cmap='gray')
                plt.imshow(img2[x_s:x_e,y_s:y_e,start_slice+i]>0.1, cmap='plasma', alpha=0.25)  
                plt.title(str(i), fontsize=40)
    else:
            for i in range(number_slices):
                fig.add_subplot(number_slices,2,2*i+1)
                plt.imshow(img1[x_s:x_e,y_s:y_e,start_slice+i], cmap='gray')
                fig.add_subplot(number_slices,2,2*i+2)
                plt.imshow(img2[x_s:x_e,y_s:y_e,start_slice+i], cmap='gray', alpha=1)
                plt.title(str(i), fontsize=40)

## Test on HCP

In [5]:
# HCP
# Load gt mask, refined, predicition and T1w image
t1w_path='../../1_Data/1_T1w_Images_and_Labels/1_T1w_Images/HCP/208226/t1.nii.gz'
gt_path='../../1_Data/1_T1w_Images_and_Labels/4_Optic_Chiasm_Labels_Handmade_Original/HCP/208226//OC_mask_handmade_original.nii.gz'
initial_path='../../1_Data/1_T1w_Images_and_Labels/2_Optic_Chiasm_Labels_Initial/HCP/208226/OC_mask_FS.nii.gz'
refined_path='../../1_Data/1_T1w_Images_and_Labels/3_Optic_Chiasm_Labels_Refined/HCP/208226/OC_mask_refined.nii.gz'
pred_path='../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_1/HCP/208226/OC_prediction.nii.gz'


In [6]:
#CHIASM
t1w_path='../../1_Data/1_T1w_Images_and_Labels/1_T1w_Images/CHIASM/CON5/t1.nii.gz'
gt_path='../../1_Data/1_T1w_Images_and_Labels/4_Optic_Chiasm_Labels_Handmade_Original/CHIASM/CON5/OC_mask_handmade_original.nii.gz'
initial_path='../../1_Data/1_T1w_Images_and_Labels/2_Optic_Chiasm_Labels_Initial/CHIASM/CON5/OC_mask_FS.nii.gz'
refined_path='../../1_Data/1_T1w_Images_and_Labels/3_Optic_Chiasm_Labels_Refined/CHIASM/CON5/OC_mask_refined.nii.gz'
pred_path='../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_1/CHIASM/CON5/OC_prediction.nii.gz'

#pred_path='../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_1/CHIASM/CON5/OC_prediction_cropped_to_gt.nii.gz'

In [7]:
def crop_masks(gt_path, mask_path, t1w_path):
    
    # Load masks
    t1w_file = nib.load(t1w_path)
    gt_file=nib.load(gt_path)
    mask_file=nib.load(mask_path)
    
    # Transform mask to be transformed to reference space
    gt_resampled_file = resample_img(gt_file, t1w_file.affine, t1w_file.shape, 'linear')
    mask_resampled_file = resample_img(mask_file, t1w_file.affine, t1w_file.shape, 'linear')

    # Get data
    gt = gt_resampled_file.get_fdata()
    mask = mask_resampled_file.get_fdata()
    t1w = t1w_file.get_fdata()
    
    # Find bounding box
    a_min, a_max, b_min, b_max, c_min, c_max = bounding_box(gt,0,0,0)
    
    # Crop the mask to bounding box
    mask[:,:,0:c_min]=0
    mask[:,:,c_max+1:]=0
    mask[np.nonzero(mask)]=1
  
    '''
    fig = plt.figure(figsize=(20,80))
    for i in range(7):
        fig.add_subplot(10,2,2*i+1)
        plt.imshow(t1w[a_min-10:a_max+10,b_min-10:b_max+10,c_min-1+i], cmap='gray')
        plt.title('Initial')
        plt.imshow(gt[a_min-10:a_max+10,b_min-10:b_max+10,c_min-1+i]>0.1, cmap='plasma', alpha=0.4)  
        fig.add_subplot(10,2,2*i+2)
        plt.title('Refined')
        plt.imshow(t1w[a_min-10:a_max+10,b_min-10:b_max+10,c_min-1+i], cmap='gray')
        plt.imshow(mask[a_min-10:a_max+10,b_min-10:b_max+10,c_min-1+i]>0, cmap='plasma', alpha=0.4) 
    '''
    
    # Save the mask
    mask_cropped_to_gt=nib.Nifti1Image(mask, t1w_file.affine, t1w_file.header)
    nib.save(mask_cropped_to_gt, mask_path[:-7]+'_cropped_to_gt.nii.gz' ) 

In [8]:
#crop_masks(gt_path, pred_path, t1w_path)

In [9]:
CHIASM=os.listdir('../../1_Data/1_T1w_Images/CHIASM')

In [10]:
for subject in CHIASM:
    
    print(subject)
    
    # Path to ground truth mask
    gt_path='../../1_Data/2_X-mask_manual/CHIASM/'+subject+'/X-mask_manual.nii.gz'    
    
    # Path to T1w image
    t1w_path='../../1_Data/1_T1w_Images/CHIASM/'+subject+'/t1.nii.gz'
    
    mask_to_be_processed=[]

    # Path to initial 
    mask_to_be_processed.append('../../1_Data/3_X-mask_atlas-initial/CHIASM/'+subject+'/X-mask_atlas-initial_complete.nii.gz')
    
    # Path to corrected
    mask_to_be_processed.append('../../1_Data/4_X-mask_atlas-corrected/CHIASM/'+subject+'/X-mask_atlas-corrected_complete.nii.gz')
    
    # Paths to all predictions
    mask_to_be_processed+=glob.glob('../../1_Data/5_X-mask_CNN/*/*/*/CHIASM/'+subject+'/X-mask_CNN.nii.gz')
        
    # Run the script through all
    i=0
    for filename in mask_to_be_processed:
        crop_masks(gt_path, filename, t1w_path)
        i+=1

CON6
CON8
ALB1
ALB4
CON5
CON3
CHP1
ALB9
ACH1
CON2
CON1
ALB3
ALB6
ALB5
CON4
ALB7
CON7
ALB2
ALB8


In [11]:
HCP=os.listdir('../../1_Data/1_T1w_Images/HCP')

In [ ]:
for subject in HCP:
    
    print(subject)
    
    # Path to ground truth mask
    gt_path='../../1_Data/2_X-mask_manual/HCP/'+subject+'/X-mask_manual.nii.gz'    
    
    # Path to T1w image
    t1w_path='../../1_Data/1_T1w_Images/HCP/'+subject+'/t1.nii.gz'
    
    mask_to_be_processed=[]

    # Path to initial 
    mask_to_be_processed.append('../../1_Data/3_X-mask_atlas-initial/HCP/'+subject+'/X-mask_atlas-initial_complete.nii.gz')
    
    # Path to corrected
    mask_to_be_processed.append('../../1_Data/4_X-mask_atlas-corrected/HCP/'+subject+'/X-mask_atlas-corrected_complete.nii.gz')
    
    # Paths to all predictions
    mask_to_be_processed+=glob.glob('../../1_Data/5_X-mask_CNN/*/*/*/HCP/'+subject+'/X-mask_CNN.nii.gz')
        
    # Run the script through all
    i=0
    for filename in mask_to_be_processed:
        crop_masks(gt_path, filename, t1w_path)
        i+=1

151425
0 ../../1_Data/1_T1w_Images_and_Labels/2_Optic_Chiasm_Labels_Initial/HCP/151425/OC_mask_FS.nii.gz
1 ../../1_Data/1_T1w_Images_and_Labels/3_Optic_Chiasm_Labels_Refined/HCP/151425/OC_mask_refined.nii.gz
2 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_2/threshold_0.25/HCP/151425/OC_prediction.nii.gz
3 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_2/threshold_1/HCP/151425/OC_prediction.nii.gz
4 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_2/threshold_0.75/HCP/151425/OC_prediction.nii.gz
5 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_2/threshold_0.5/HCP/151425/OC_prediction.nii.gz
6 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_0.25/HCP/151425/OC_prediction.nii.gz
7 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_1/HCP/151425/OC_prediction.nii.gz
8 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/

6 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_0.25/HCP/118730/OC_prediction.nii.gz
7 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_1/HCP/118730/OC_prediction.nii.gz
8 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_0.75/HCP/118730/OC_prediction.nii.gz
9 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_3/threshold_0.5/HCP/118730/OC_prediction.nii.gz
10 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_1/threshold_0.25/HCP/118730/OC_prediction.nii.gz
11 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_1/threshold_1/HCP/118730/OC_prediction.nii.gz
12 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_1/threshold_0.75/HCP/118730/OC_prediction.nii.gz
13 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_1/threshold_0.5/HCP/118730/OC_prediction.nii.gz
14 ../../1_Data/2_DC

12 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_1/threshold_0.75/HCP/165436/OC_prediction.nii.gz
13 ../../1_Data/2_DCNN_Predictions/training_30ep_00025lr_dice/connectivity_1/threshold_0.5/HCP/165436/OC_prediction.nii.gz
14 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_2/threshold_0.25/HCP/165436/OC_prediction.nii.gz
15 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_2/threshold_1/HCP/165436/OC_prediction.nii.gz
16 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_2/threshold_0.75/HCP/165436/OC_prediction.nii.gz
17 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_2/threshold_0.5/HCP/165436/OC_prediction.nii.gz
18 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_3/threshold_0.25/HCP/165436/OC_prediction.nii.gz
19 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_3/threshold_1/HCP/165436/OC_prediction.nii.gz
20 ../../1_Data/

17 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_2/threshold_0.5/HCP/134223/OC_prediction.nii.gz
18 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_3/threshold_0.25/HCP/134223/OC_prediction.nii.gz
19 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_3/threshold_1/HCP/134223/OC_prediction.nii.gz
20 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_3/threshold_0.75/HCP/134223/OC_prediction.nii.gz
21 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_3/threshold_0.5/HCP/134223/OC_prediction.nii.gz
22 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_1/threshold_0.25/HCP/134223/OC_prediction.nii.gz
23 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_1/threshold_1/HCP/134223/OC_prediction.nii.gz
24 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_1/threshold_0.75/HCP/134223/OC_prediction.nii.gz
25 ../../1_Data/

22 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_1/threshold_0.25/HCP/673455/OC_prediction.nii.gz
23 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_1/threshold_1/HCP/673455/OC_prediction.nii.gz
24 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_1/threshold_0.75/HCP/673455/OC_prediction.nii.gz
25 ../../1_Data/2_DCNN_Predictions/training_13ep_00025lr_dice/connectivity_1/threshold_0.5/HCP/673455/OC_prediction.nii.gz
26 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_2/threshold_0.25/HCP/673455/OC_prediction.nii.gz
27 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_2/threshold_1/HCP/673455/OC_prediction.nii.gz
28 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_2/threshold_0.75/HCP/673455/OC_prediction.nii.gz
29 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_2/threshold_0.5/HCP/673455/OC_prediction.nii.gz
30 ../../1_D

27 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_2/threshold_1/HCP/304727/OC_prediction.nii.gz
28 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_2/threshold_0.75/HCP/304727/OC_prediction.nii.gz
29 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_2/threshold_0.5/HCP/304727/OC_prediction.nii.gz
30 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_3/threshold_0.25/HCP/304727/OC_prediction.nii.gz
31 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_3/threshold_1/HCP/304727/OC_prediction.nii.gz
32 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_3/threshold_0.75/HCP/304727/OC_prediction.nii.gz
33 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_3/threshold_0.5/HCP/304727/OC_prediction.nii.gz
34 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_1/threshold_0.25/HCP/304727/OC_prediction.nii.gz
35 ../..

32 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_3/threshold_0.75/HCP/101107/OC_prediction.nii.gz
33 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_3/threshold_0.5/HCP/101107/OC_prediction.nii.gz
34 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_1/threshold_0.25/HCP/101107/OC_prediction.nii.gz
35 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_1/threshold_1/HCP/101107/OC_prediction.nii.gz
36 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_1/threshold_0.75/HCP/101107/OC_prediction.nii.gz
37 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_1/threshold_0.5/HCP/101107/OC_prediction.nii.gz
38 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_2/threshold_0.25/HCP/101107/OC_prediction.nii.gz
39 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_2/threshold_1/HCP/101107/OC_prediction.nii.gz
40 ../../1

37 ../../1_Data/2_DCNN_Predictions/training_100ep_00005lr_dice/connectivity_1/threshold_0.5/HCP/208226/OC_prediction.nii.gz
38 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_2/threshold_0.25/HCP/208226/OC_prediction.nii.gz
39 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_2/threshold_1/HCP/208226/OC_prediction.nii.gz
40 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_2/threshold_0.75/HCP/208226/OC_prediction.nii.gz
41 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_2/threshold_0.5/HCP/208226/OC_prediction.nii.gz
42 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_3/threshold_0.25/HCP/208226/OC_prediction.nii.gz
43 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_3/threshold_1/HCP/208226/OC_prediction.nii.gz
44 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_3/threshold_0.75/HCP/208226/OC_prediction.nii.gz
45 ../../1_Data

42 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_3/threshold_0.25/HCP/131823/OC_prediction.nii.gz
43 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_3/threshold_1/HCP/131823/OC_prediction.nii.gz
44 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_3/threshold_0.75/HCP/131823/OC_prediction.nii.gz
45 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_3/threshold_0.5/HCP/131823/OC_prediction.nii.gz
46 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_1/threshold_0.25/HCP/131823/OC_prediction.nii.gz
47 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_1/threshold_1/HCP/131823/OC_prediction.nii.gz
48 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_1/threshold_0.75/HCP/131823/OC_prediction.nii.gz
49 ../../1_Data/2_DCNN_Predictions/training_40ep_00015lr_dice/connectivity_1/threshold_0.5/HCP/131823/OC_prediction.nii.gz
50 ../../1_Data/